In [1]:
import torch
from datasets import load_dataset
from datasets import load_from_disk

# from d2l import torch as d2l


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # dataset = load_dataset('lansinuote/ChnSentiCorp', keep_in_memory=True)
        dataset = load_from_disk('./data/ChnSentiCorp')
        
        print(dataset)
        def f(data):
            return len(data['text']) > 30
        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        return text

    
dataset = Dataset('train')
len(dataset) # , dataset[0]

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/train/cache-477de4adc0b5b333.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/test/cache-2942213d0d889635.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/validation/cache-83eb6d135357ad0e.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})


3

In [50]:
# dataset['test']
dataset

In [3]:
from transformers import BertTokenizer

# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

token

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [18]:
def collate_fn(data):
    
    # print(data)
    print('data长度: ', len(data))
    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=30,
                                   return_tensors='pt',
                                   return_length=True)

    print('编码后的data打印: ', data.keys())
    
    # input_ids:编码之后的数字
    # attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    # 把第15个词固定替换为mask
    print('第15个词: %s' % input_ids[:, 15])
    
    # 这里直接使用了编码后的数据作为真实预测值
    labels = input_ids[:, 15].reshape(-1).clone()
    input_ids[:, 15] = token.get_vocab()[token.mask_token]

    # print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

## 数据加载器

In [19]:
# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset['train'],
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True
                                    )


for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print(len(loader))
    print(token.decode(input_ids[0]))
    print(token.decode(labels[0]))
    print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
    print("")
    
    if i >= 5:
        break

data长度:  16
编码后的data打印:  dict_keys(['input_ids', 'token_type_ids', 'length', 'attention_mask'])
第15个词: tensor([6163, 2523, 4500, 6235, 5291, 6963,  749, 3177, 5010, 5450, 4638, 8024,
        6843, 2190, 2382, 8111])
574
[CLS] 整 体 还 可 以 ， 就 是 不 配 系 统 ， 安 [MASK] [UNK] 系 统 不 方 便 ， 换 了 几 个 系 统 [SEP]
装
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

data长度:  16
编码后的data打印:  dict_keys(['input_ids', 'token_type_ids', 'length', 'attention_mask'])
第15个词: tensor([ 122,  671, 8039, 3949, 3198, 6772,  749,  131, 6981, 4684, 4669, 3189,
        4192, 4638,  117, 6821])
574
[CLS] 二 年 前 给 他 买 了 全 套 的 神 奇 校 车 [MASK] ， 这 套 书 成 了 他 最 喜 欢 看 书 之 [SEP]
1
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

data长度:  16
编码后的data打印:  dict_keys(['input_ids', 'token_type_ids', 'length', 'attention_mask'])
第15个词: tensor([ 738, 1104, 1501, 1213,  679,  100, 6772, 7028, 1391, 1963, 1380, 4638,
        8024, 1057, 8024, 2382])
574
[CLS] 每 个 人 的 看 法 可 能 都 

## 加载预训练模型

In [6]:
from transformers import BertModel


# 加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese').to(device)


# 不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)


def test_1():
    # 模型试算
    out = pretrained(input_ids=input_ids,
                     attention_mask=attention_mask,
                     token_type_ids=token_type_ids
                    )

    out.last_hidden_state.shape
    pass

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 定义下游任务模型

In [7]:

# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.decoder = torch.nn.Linear(768, token.vocab_size, bias=False)
        self.bias = torch.nn.Parameter(torch.zeros(token.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, input_ids, attention_mask, token_type_ids):
        
        # pretrained = pretrained.to(device)

        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)
            pass
        
        out = self.decoder(out.last_hidden_state[:, 15])
        return out


model = Model()


#model(input_ids=input_ids,
#      attention_mask=attention_mask,
#      token_type_ids=token_type_ids
#     ).shape


# 模型转移到GPU上
model.to(device)

Model(
  (decoder): Linear(in_features=768, out_features=21128, bias=True)
)

In [19]:
list(model.parameters())[0].device

device(type='cuda', index=0)

## 训练

In [ ]:
from transformers import AdamW


# 训练
optimizer = AdamW(model.parameters(), lr=5e-4)

# 损失函数
loss = torch.nn.CrossEntropyLoss()


model.train()


print("training on ", device)
for epoch in range(5):
    
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
        # print('labels', labels)
        # labels tensor([2523, 1962, ....,  6817, 1962])
        
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)
        
        # print(input_ids)
        
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids
                   )
        
        # print('out: ', out, out.shape)  torch.Size([16, 21128])
        # print('labels: ', labels, labels.shape)  torch.Size([16])
        # print(out.cpu())

        l = loss(out, labels)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()

        if i % 100 == 0:
            out = out.cpu()
            labels = labels.cpu()
            
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)
            print('训练批次: %s 当前第 %s 轮训练 loss: %s acc: %s' % (
                        epoch, i, l.item(), accuracy)
                 )

In [44]:
#测试
def test_calculate():
    
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=dataset['test'],
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        
        if i == 15:
            break

        with torch.no_grad():
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)
            
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
            pass
        
        out = out.argmax(dim=1).cpu()
        labels = labels.cpu()
        
        correct += (out == labels).sum().item()
        total += len(labels)

        print('序: %s 输入内容: %s' % (i, token.decode(input_ids[0])))
        print('解码: ', token.decode(labels[0]), token.decode(labels[0]))
        
    print('acc: %.2f' % (correct / total))
    pass

In [45]:
# 预测
test_calculate()

序: 0 输入内容: [CLS] 跟 很 多 买 家 一 样 遇 到 了 表 面 金 属 [MASK] 牌 划 痕 问 题 ， 但 知 道 没 法 退 货 [SEP]
解码:  标 标
序: 1 输入内容: [CLS] 外 观 跟 我 想 象 的 不 一 样 ， 是 康 博 [MASK] 电 脑 ， 一 查 原 来 也 属 于 [UNK] 。 没 [SEP]
解码:  的 的
序: 2 输入内容: [CLS] 房 间 很 大 味 道 ， 服 务 差 ， 态 度 差 [MASK] 房 单 设 施 差 ~ ~ 永 远 不 会 再 去 [SEP]
解码:  ， ，
序: 3 输入内容: [CLS] 1. 电 池 时 间 长 ， 普 通 使 用 5 - [MASK] 小 时 没 问 题 ， 省 点 用 可 以 到 近 [SEP]
解码:  6 6
序: 4 输入内容: [CLS] 1 、 打 车 到 宾 馆 和 离 开 ， 送 上 登 [MASK] 出 租 车 牌 号 的 卡 片 ， 如 有 物 品 [SEP]
解码:  记 记
序: 5 输入内容: [CLS] 书 送 来 的 时 候 刚 好 是 六 一 儿 童 节 [MASK] 天 的 中 午 ， 儿 子 见 到 书 之 后 迅 [SEP]
解码:  那 那
序: 6 输入内容: [CLS] 太 重 ， 装 系 统 只 能 装 [UNK] 的 ， 我 的 [MASK] 版 xp3 都 不 能 装 ， 搞 了 我 半 天 [SEP]
解码:  正 正
序: 7 输入内容: [CLS] 不 能 装 锁 、 光 驱 读 盘 能 力 不 算 太 [MASK] 、 圆 通 快 递 太 离 谱 ， 到 货 无 通 [SEP]
解码:  好 好
序: 8 输入内容: [CLS] flash 分 编 程 和 动 画 两 部 分 ， 这 本 书 [MASK] 的 是 flash [UNK] ， 可 是 脚 本 的 运 用 任 [SEP]
解码:  说 说
序: 9 输入内容: [CLS] 住 这 个 酒 店 实 在 是 太 享 受 了, 不 [MASK] 可 以 使 用 五 彩 缤 纷 的 白 毛 巾, [SEP]
解码:  仅 仅
序: 10 输入内容: [CLS] 性 价 比 高 ， led 屏 幕 ， 西 数 的 硬 盘 [MASK